In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [2]:
public = pd.read_csv('./data/Public_Housing_Buildings.csv')
public.columns = [x.lower() for x in public.columns]

C:\Users\jackv\AppData\Local\Temp\ipykernel_1264\2237559633.py:1: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  public = pd.read_csv('./data/Public_Housing_Buildings.csv')


In [3]:
len(public)

191885

In [4]:
public.sample()

,x,y,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email
156414,-98.1514,29.135141,156415,TX193,Floresville Housing Authority,TX193000001,NONE,None,5,SD,INAPCP,4.649420e+09,1963/06/30 00:00:00+00,1963/03/31 00:00:00+00,2,2,2,0,2,100.0,2,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,48.0,Wilson,493.0,300.0,3.0,3057.0,NaN,NaN,Wilson,493.0,91330.0,Floresville,26160.0,Floresville city,C1,NaN,7240.0,"San Antonio, TX MSA",41700.0,"San Antonio-New Braunfels, TX Metropolitan Sta...",99999.0,NaN,1.0,0.0,4828,1818.0,NaN,NaN,1512 E St,NaN,Floresville,TX,78114.0,2332.0,NaN,78114.0,12.0,6.0,7.811423e+10,S,NaN,NaN,NaN,1.0,NaN,NaN,A,NaN,90% Overall probable correctness,29.135141,-98.1514,S,NaN,R,U,** Street-Level Rooftop **,48493.0,4826160.0,4.849300e+10,4.849300e+11,N,,2023/03/06 08:50:52+00,-4,-4.0,40,TX19300000151,8303936560,8.303936e+09,floresvilleha@gmail.com,8.303937e+09,NaN,floresvilleha@gmail.com


In [5]:
dev = pd.read_csv('./data/Public_Housing_Developments.csv')
dev.columns = [x.lower() for x in dev.columns]
dev = dev[dev.project_name.isin(public.project_name)]

In [6]:
dev.sample()

,x,y,objectid,participant_code,formal_participant_name,development_code,project_name,scattered_site_ind,pd_status_type_code,total_units,total_dwelling_units,acc_units,total_occupied,regular_vacant,pha_total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,spending_per_month_prev_yr,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email
4961,-97.217847,26.066482,4962,TX173,Port Isabel Housing Authority,TX173200001,VILLA DEL MAR,N,M,80,80,80,80,0,80,100.0,79,98.75,7,20.25,3.3,263,396,567,15645,4700,1.27,25.32,27.85,24.05,21.52,67.09,0.0,32.91,34.07,94.94,60.76,36.71,24.05,67.09,44.3,12.07,47.62,6.46,0.0,56.96,16.46,26.58,1.27,94.94,0.0,0.0,0.0,94.94,0,84,100.0,108,25.32,17.72,56.96,2.53,77.36,25.75,50.61,48.0,Cameron,61.0,12304.0,2.0,2078.0,N,N,Cameron,61.0,91115.0,East Cameron,58892.0,Port Isabel city,C1,NaN,1240.0,"Brownsville-Harlingen-San Benito, TX MSA",15180.0,"Brownsville-Harlingen, TX Metropolitan Statist...",99999.0,NaN,1.0,0.0,4834,4254.0,Y,NaN,506 Port Rd Apt A,NaN,Port Isabel,TX,78578.0,3714.0,NaN,78578.0,73.0,0.0,7.857837e+10,H,A,Apt,NaN,2.0,NaN,NaN,NaN,NaN,80% Overall probable correctness,26.066482,-97.217847,S,NaN,R,U,** Street-Level Rooftop **,48061.0,4858892.0,4.806101e+10,4.806100e+11,559,126,9.51,2.66,100.0,14136,2022/01/06 20:00:29+00,544396,523398.0,9569432863,9.569436e+09,jmsexdrJ@netscape.net,9.569433e+09,NaN,elzslns@netscape.net


In [7]:
dev_to_add = dev.loc[:, [
    "development_code",
    "spending_per_month_prev_yr",
    "chldrn_mbr_cnt",
    "annl_expns_amnt",
    "annl_expns_amnt_prev_yr",
    "eldly_prcnt",
    "pct_disabled_lt62_all",
    "pct_lt80_median",
    "median_inc_amnt",
    "scattered_site_ind",
    "pd_status_type_code",
    "total_units",
    "total_dwelling_units",
    "acc_units",
    "total_occupied",
    "regular_vacant",
    "pha_total_units",
    "pct_occupied",
    "number_reported",
    "pct_reported",
    "months_since_report",
    "pct_movein",
    "people_per_unit",
    "people_total",
    "rent_per_month",
    "spending_per_month",
    "hh_income",
    "person_income",
    "pct_lt5k",
    "pct_5k_lt10k",
    "pct_10k_lt15k",
    "pct_15k_lt20k",
    "pct_ge20k",
    "pct_wage_major",
    "pct_welfare_major",
    "pct_other_major",
    "pct_median",
    "pct_lt50_median",
    "pct_lt30_median",
    "pct_2adults",
    "pct_1adult",
    "pct_female_head",
    "pct_female_head_child",
    "pct_disabled_lt62",
    "pct_disabled_ge62",
    "pct_disabled_all",
    "pct_lt24_head",
    "pct_age25_50",
    "pct_age51_61",
    "pct_age62plus",
    "pct_age85plus",
    "pct_minority",
    "pct_black",
    "pct_native_american",
    "pct_asian",
    "pct_hispanic",
    "months_waiting",
    "months_from_movein",
    "pct_utility_allow",
    "ave_util_allow",
    "pct_bed1",
    "pct_bed2",
    "pct_bed3",
    "pct_overhoused",
    "tminority",
    "tpoverty",
    "tpct_ownsfd"
]].copy()
dev_to_add.columns = ['dev_' + str(x) for x in dev_to_add.columns]

In [8]:
public.development_code = public.development_code.str.upper()
dev.dev_development_code = dev_to_add.dev_development_code.str.upper()

C:\Users\jackv\AppData\Local\Temp\ipykernel_13856\779776153.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dev.dev_development_code = dev_to_add.dev_development_code.str.upper()


In [9]:
public = public.merge(dev_to_add, left_on='development_code', 
             right_on='dev_development_code', how='left')   

In [10]:
len(public)

191885

In [11]:
pha = pd.read_csv('./data/Public_Housing_Authorities.csv')
pha.columns = [x.lower() for x in pha.columns]

In [12]:
public.sample()

,x,y,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd
107627,-97.881439,27.530474,107628,TX114,Kingsville Housing Authority,TX114000002,The Heights at Corral,Heights,02-217,RW,INAPCP,8.308030e+09,2011/07/31 00:00:00+00,2011/08/23 00:00:00+00,1,0,0,0,1,0.0,0,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,48.0,Kleberg,273.0,20300.0,1.0,1014.0,NaN,NaN,Kleberg,273.0,92035.0,Kingsville,39352.0,Kingsville city,C1,NaN,9999.0,NaN,28780.0,"Kingsville, TX Micropolitan Statistical Area",99999.0,NaN,0.0,1.0,4834,2826.0,NaN,NaN,1000 W Corral Ave,NaN,Kingsville,TX,78363.0,3057.0,NaN,78363.0,99.0,0.0,7.836331e+10,H,1SUD,NaN,NaN,2.0,NaN,N,N,NaN,80% Overall probable correctness,27.530474,-97.881439,S,NaN,R,U,** Street-Level Rooftop **,48273.0,4839352.0,4.827302e+10,4.827302e+11,N,,2023/03/06 08:50:52+00,-4,-4.0,95,TX11400000

In [13]:
pha_to_add = pha.loc[:, [
    'participant_code', 
    'phas_designation',
    'ha_low_rent_size_category',
    'section8_units_cnt',
    'ha_section_8_size_category',
    'ha_combined_size_category',
    'ha_fye',
    'ha_program_type',
    'section8_occupied',
    'opfund_amnt',
    'opfund_amnt_prev_yr',
    'capfund_amnt',
    'capfund_amnt_prev_yr',
    'ross_amnt',
    'fss_amnt',
    'chldrn_mbr_cnt',
    'eldly_prcnt',
    'pct_disabled_lt62_all',
    'pct_lt80_median',
    'median_inc_amnt',
    'annl_expns_amnt',
    'annl_expns_amnt_prev_yr'
]].copy()
pha_to_add.columns = ['pha_' + str(x) for x in pha_to_add.columns]

In [14]:
pha_to_add.pha_participant_code = pha_to_add.pha_participant_code.str.upper()
public.participant_code = public.participant_code.str.upper()

In [15]:
public = public.merge(pha_to_add, left_on='participant_code', 
             right_on='pha_participant_code', how='left')   

In [16]:
len(public)

191885

In [17]:
public['std_addr'] = public.std_addr.str.upper()
public['std_st'] = public.std_st.str.upper()
public['std_city'] = public.std_city.str.upper()

In [18]:
def correct_address(std_addr):
    if pd.isna(std_addr):
        return np.nan, np.nan
    if 'APT' in std_addr:
        split_addr = std_addr.split('APT')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'UNIT' in std_addr:
        split_addr = std_addr.split('UNIT')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'SUB' in std_addr:
        split_addr = std_addr.split('SUB')
        return split_addr[0].strip(), split_addr[1].strip()
    if '#' in std_addr:
        split_addr = std_addr.split('#')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'LOT' in std_addr:
        split_addr = std_addr.split('LOT')
        return split_addr[0].strip(), split_addr[1].strip()
    return std_addr, np.nan

public[['corrected_address', 'corrected_unit']] = public.std_addr.apply(correct_address).apply(pd.Series)

In [19]:
public['corrected_address'] = public.corrected_address.str.replace('OFC', '')
public['corrected_address'] = public.corrected_address.str.replace('1SUD', '')

In [20]:
public.corrected_unit = public.corrected_unit.fillna('')

In [21]:
public = public.drop(['apt_no'], axis=1)

Process NYC city names

In [22]:
public[public.std_st == 'NY'].groupby('std_city').size().sort_values(ascending=False).head(15)

std_city
BROOKLYN         720
NEW YORK         583
BUFFALO          560
ROCHESTER        548
BRONX            518
QUEENS           206
SYRACUSE         176
NIAGARA FALLS    171
ALBANY           136
UTICA             85
STATEN ISLAND     78
SCHENECTADY       77
PLATTSBURGH       68
KINGSTON          59
ELMIRA            54
dtype: int64

In [23]:
public.std_city = public.std_city.str.replace('STATEN ISLAND', 'NEW YORK')
public.std_city = public.std_city.str.replace('QUEENS', 'NEW YORK')
public.std_city = public.std_city.str.replace('BRONX', 'NEW YORK')
public.std_city = public.std_city.str.replace('MANHATTAN', 'NEW YORK')
public.std_city = public.std_city.str.replace('BROOKLYN', 'NEW YORK')
public.std_city = public.std_city.str.replace('NEW YORK CITY', 'NEW YORK')

In [24]:
public[public.std_st == 'NY'].groupby('std_city').size().sort_values(ascending=False).head(15)

std_city
NEW YORK            2116
BUFFALO              560
ROCHESTER            548
SYRACUSE             176
NIAGARA FALLS        171
ALBANY               136
UTICA                 85
SCHENECTADY           77
PLATTSBURGH           68
KINGSTON              59
ELMIRA                54
LOCKPORT              46
WATERTOWN             45
SARATOGA SPRINGS      43
BINGHAMTON            42
dtype: int64

In [25]:
public.sample()

,x,y,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_participant_code,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit
26451,-87.061073,31.128536,26452,AL156,Housing Authority of the City of Brewton,AL156000002,BRYANT CIRCLE,NaN,011,SD,INAPCP,4.404100e+09,1962/01/01 00:00:00+00,1962/09/01 00:00:00+00,2,2,2,0,2,100.0,2,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,1.0,Escambia,53.0,970200

In [26]:
public.to_csv('buildings_dev_pha_2.1.csv', index=False)